<a href="https://colab.research.google.com/github/ephipie/human-ai-parallel-detection/blob/main/LLM_Detection_02_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -q

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
import pandas as pd
import numpy as np

In [ ]:
model = SentenceTransformer('AnnaWegmann/Style-Embedding')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

working_df = pd.read_parquet("/content/drive/MyDrive/shared_data/llm_detection_data.parquet")

In [ ]:
%%time

TEXT_COLS_TO_EMB  = ["chunk_1", "chunk_2", "gpt", "llama"]
BATCH             = 64
PROGRESS_BAR      = True

def compute_col_embeddings(df,columns = TEXT_COLS_TO_EMB,batch_size=BATCH,progress_bar = PROGRESS_BAR):
  for col in columns:
      # Prepare a clean list of strings (empty string -> 0-vector).
      texts     = df[col].fillna("").tolist()

      # Batch-compute embeddings in one shot.
      embs      = model.encode(
          texts,
          batch_size        = batch_size,
          show_progress_bar = progress_bar,
          convert_to_numpy  = True,
          normalize_embeddings = False   #
      )
      # Write back as a new column
      df[f"{col}_embeddings"] = list(embs)


compute_col_embeddings(working_df)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Save
working_df.to_parquet('/content/drive/MyDrive/shared_data/llm_detection_data_embeds.parquet')